

https://medium.com/mlearning-ai/all-8-types-of-time-series-classification-methods-2c8e4b323ea2#bd12

For any usuall classsification problem, we need to have some set of features, but in our case we
have different time series, so our first taks is to convert these time series into features. 

Transformations: 
- Lienar transformations using PCA: As taken from different thoughts from
[here](https://www.linkedin.com/advice/0/what-advantages-disadvantages-using-pca), we are ingnoring
exploring PCA on time series.
- Non-Linear transformations like T-SNE: 
  - T-SNE is a technique used for visual the data, it does not learn any function from the original
  space to the new (lower) dimensional one. so this technique is not sutibale for feature
  extraction. Please refer the discussion
  [here](https://stats.stackexchange.com/questions/340175/why-is-t-sne-not-used-as-a-dimensionality-reduction-technique-for-clustering-or)
  - Auto encoders are a good options, we will do this in later sections.
- Latent Space Feaure extractions
  - tsfresh
  - pyts
  - cesium
  

Note: 
- Please be aware, tsfresh feature extraction is a bit slow.
- We are using only default setting for feature extraction, Please refere documentation for details and advanced settings for feature extraction.

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
from src.hydraulic.config import DATA_PATH, SEPERATOR, PROFILE_COLUMNS, FILES_NAMES, DATA_OUTPUT_PATH
from tslearn.utils import to_sktime_dataset

In [4]:
target = pd.read_csv(F'{DATA_PATH}/profile.txt',sep=SEPERATOR, names=PROFILE_COLUMNS)
print(f"Target data array shape : {target.shape}")

# Load data and converting data into TS arrays
sktime_df = pd.DataFrame()
for sensor_type, data_file_names_list in FILES_NAMES.items():
    for file_name in data_file_names_list:
        if sensor_type in ['pressure', 'motor_power', 'volume_flow']:
            pass
        else:
            # read txt file
            ts_data = pd.read_csv(f'./data/{file_name}.txt', sep='\t', header=None)
            print(f"{sensor_type} - {file_name} : {ts_data.shape}")
            # transform column data to TS array for TSKmeansClustring.
            sktime_df[f"{sensor_type}_{file_name}"] = to_sktime_dataset(ts_data)

Target data array shape : (2205, 5)
temperature - TS1 : (2205, 60)
temperature - TS2 : (2205, 60)
temperature - TS3 : (2205, 60)
temperature - TS4 : (2205, 60)
vibration - VS1 : (2205, 60)
cooling_efficiency - CE : (2205, 60)
cooling_power - CP : (2205, 60)
efficiency_factor - SE : (2205, 60)


### Data splitting

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_idx, test_idx = train_test_split(sktime_df.index, random_state=9)

X_train, y_train = sktime_df.loc[train_idx], target.loc[train_idx]
X_test, y_test = sktime_df.loc[test_idx], target.loc[test_idx]

In [6]:
print(f"Train X Shape: {X_train.shape}")
print(f"Train y Shape: {y_train.shape}")
print(f"Sample Train X:")
X_train.head(2)

Train X Shape: (1653, 8)
Train y Shape: (1653, 5)
Sample Train X:


,temperature_TS1,temperature_TS2,temperature_TS3,temperature_TS4,vibration_VS1,cooling_efficiency_CE,cooling_power_CP,efficiency_factor_SE
246,0 54.414 1 54.340 2 54.309 3 5...,0 58.887 1 58.871 2 58.871 3 5...,0 55.754 1 55.754 2 55.762 3 5...,0 49.484 1 49.496 2 49.445 3 4...,0 0.653 1 0.688 2 0.711 3 0.72...,0 20.376 1 20.551 2 20.483 3 2...,0 1.555 1 1.560 2 1.566 3 1.58...,0 65.230 1 0.000 2 0.000 3 ...
847,0 46.793 1 46.727 2 46.621 3 4...,0 51.844 1 51.816 2 51.797 3 5...,0 48.848 1 48.859 2 48.895 3 4...,0 42.328 1 42.320 2 42.254 3 4...,0 0.591 1 0.626 2 0.670 3 0.68...,0 26.098 1 26.055 2 26.375 3 2...,0 1.697 1 1.698 2 1.715 3 1.70...,0 68.879 1 0.000 2 0.000 3 ...


In [7]:
print(f"Sample Train Y:")
y_train.head(2)

Sample Train Y:


,cooler,valve,pump_lekage,accumulator,stable_flag
246,3,100,2,130,0
847,20,100,0,90,1


In [8]:
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier

# kind of feature selection
from sktime.transformations.panel.channel_selection import ElbowClassPairwise

# feature transfomers 
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor
from sktime.transformations.panel.catch22 import Catch22


Current transformers accepts all sensons should be of same size. so excluding some sensors in the
model. Try out different transfamations to find out best performance of the final model.

Usually channel selection makes training and infrence processing much faster.


### ROCKET Transformations

In [9]:
# excluding sensors with have time series lent is not equal to 60
exclude_sensors = [i for i in X_train.columns if i.split('_')[0] in ['pressure', 'motor', 'volume']]

In [10]:
rocket_pipeline = MultiOutputClassifier(
    make_pipeline(
        ElbowClassPairwise(), # channel selection, kind of seansor selection for training
        Rocket(num_kernels=10000, n_jobs=-1, random_state=200), # Feature extranction transformations
        RandomForestClassifier(random_state=200)  # regular random forest classifier
    )
)

rocket_pipeline.fit(X_train, y_train)
test_predictions_ = rocket_pipeline.predict(X_test)
for i, v in enumerate(y_test.columns):
    print(f"Model Accuracy for {v} : {accuracy_score(y_test[v].values,test_predictions_[:,i])}")

Model Accuracy for cooler : 0.9909420289855072
Model Accuracy for valve : 0.8894927536231884
Model Accuracy for pump_lekage : 0.7934782608695652
Model Accuracy for accumulator : 0.8822463768115942
Model Accuracy for stable_flag : 0.9094202898550725


In [11]:
catch22_pipeline = MultiOutputClassifier(
    make_pipeline(
        ElbowClassPairwise(), # channel selection, kind of seansor selection for training
        Catch22(n_jobs=-1, catch24=True), # Feature extranction transformations
        RandomForestClassifier(random_state=200)  # regular random forest classifier
    ) 
)

catch22_pipeline.fit(X_train, y_train)
test_predictions_ = catch22_pipeline.predict(X_test)
for i, v in enumerate(y_test.columns):
    print(f"Model Accuracy for {v} : {accuracy_score(y_test[v].values, test_predictions_[:,i])}")

Model Accuracy for cooler : 0.9981884057971014
Model Accuracy for valve : 0.8768115942028986
Model Accuracy for pump_lekage : 0.9818840579710145
Model Accuracy for accumulator : 0.9655797101449275
Model Accuracy for stable_flag : 0.9293478260869565


### TSFresh transformations

Randonly this transformations are not working while in prediction part. So need to validate this.

In [12]:

tsfresh_pipeline = MultiOutputClassifier(
    make_pipeline(
        ElbowClassPairwise(), # channel selection, kind of seansor selection for training
        TSFreshFeatureExtractor(
            default_fc_parameters = "efficient",
            show_warnings=False,
            n_jobs=-1
        ), # Feature extranction transformations
        RandomForestClassifier(random_state=200)  # regular random forest classifier
    )
)

tsfresh_pipeline.fit(X_train, y_train)
test_predictions_ = tsfresh_pipeline.predict(X_test)
for i, v in enumerate(y_test.columns):
    print(f"Model Accuracy for {v} : {accuracy_score(y_test[v].values, test_predictions_[:,i])}")

Feature Extraction: 100%|██████████| 60/60 [00:07<00:00,  8.09it/s]


ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.
